```markdown
# Loading and Checking the Dataset

In this section, we will load the dataset and perform an initial check to understand its structure and contents. This includes importing necessary libraries, creating a sample dataframe, and displaying the dataframe to verify the data.
```

In [1]:
# Import Libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
from dateparser import parse
from tqdm import tqdm




In [27]:

# Load Data
df = pd.read_csv(r'E:\Programming\Python\Proyek Scraping\olx\rumah\jabodetabeksur_olx_housing_dataset.csv')

# Menampilkan daftar unik pada kolom 'address_district'
unique_districts = df['address_district'].unique()

# Menampilkan hasil
print(unique_districts)

['Pancoran Mas' 'Sawangan' 'Citayam' 'Bojongsari' 'Limo' 'Cipayung'
 'Senen' 'Cibinong' 'Beji' 'Sukmajaya' 'Tapos' 'Cimanggis' 'Cibubur'
 'Jagakarsa' 'Ciracas' 'Pasar Rebo' 'Cilangkap' 'Gunung Putri'
 'Jati Sampurna' 'Kalisari' 'Cilodong' 'Bojonggede' 'Pamulang'
 'Ciputat Timur' 'Cinere' 'Gunung Sindur' 'Parung' 'Cisauk' 'Tajurhalang'
 'Kemang' 'Bekasi Barat' 'Medan Satria' 'Bekasi Jaya' 'Bekasi Timur'
 'Bekasi Utara' 'Cakung' 'Tarumajaya' 'Pondok Gede' 'Jatiasih' 'Jatiwarna'
 'Pondok Melati' 'Bambu Apus' 'Bekasi Selatan' 'Bantar Gebang'
 'Jatimakmur' 'Rawalumbu' 'Jatimulya' 'Mustika Jaya' 'Cileungsi'
 'Duren Sawit' 'Makasar' 'Sawah Besar' 'Tambun Selatan' 'Cikarang Barat'
 'Setu' nan 'Bogor Barat - Kota' 'Tanah Sareal' 'Bogor Utara - Kota'
 'Ciomas' 'Bogor Timur - Kota' 'Tanah Sereal' 'Bumi Serpong Damai'
 'Bogor Tengah - Kota' 'Cikarang Selatan' 'Parung Panjang' 'Ciputat'
 'Dramaga' 'Sukaraja' 'Bogor Selatan - Kota' 'Cijeruk' 'Tamansari'
 'Babakan Madang' 'Megamendung' 'Pasar Minggu'

```markdown
# Fixing Numerical Type Columns

This section address and correcting the data types of several numerical columns in our dataset. These columns include:

- `bedrooms`
- `bathrooms`
- `maid_bedrooms`
- `maid_bathrooms`
- `ruang_tamu`
- `ruang_makan`
- `additional_rooms`
- `floors`

Ensuring these column's value makes sense while also changing their type to float.
```

In [23]:
df.loc[df['type'] == 'Apartemen', 'floors'] = 1.0

df['maid_bathrooms'] = df['maid_bathrooms'].astype(float)
df['maid_bedrooms'] = df['maid_bedrooms'].astype(float)
df['ruang_makan'] = df['ruang_makan'].astype(float)
df['ruang_tamu'] = df['ruang_tamu'].astype(float)
df['additional_rooms'] = df['additional_rooms'].fillna(0).astype(int)
df['floors'] = df['floors'].astype(float)

output_file = "step1_jabodetabeksur_olx_housing_dataset_.csv"
df.to_csv(output_file, index=False)




```Markdown
# First Step of Exctracting Information

In this step the goal is to extract information from description and facilities column to fill and/or fix missing value in the floors, additional_rooms, certificate, garage_capacity, carport_capacity, electricity_capacity, and house_orientation column.
```

In [24]:
# Function to extract information from descriptions
def scrape_description(ads_description, kata_kunci_list, entity_type=None):
    headings = [
        "Timur", "Tenggara", "Selatan", "Barat Daya", "Barat", "Barat Laut", 
        "Utara", "Timur Laut"
    ]
    
    if not isinstance(ads_description, list):
        return None
    
    for kata_kunci in kata_kunci_list:
        if entity_type == 'electricity':
            pattern = rf'{re.escape(kata_kunci)}\s*[\:\.\-\s]*([\d.,]+)\s*(watt|va|kva|token|w|wt|kwh)?'
        elif entity_type in ['garage', 'carport']:
            pattern = rf'{re.escape(kata_kunci)}\s*[\:\.\-\s]*(\d+)?\s*(mobil|mbl|cars?)?'
        elif entity_type == 'heading':
            pattern = rf'{re.escape(kata_kunci)}\s*[\:\.\-\s]*(.*?)(Timur|Tenggara|Selatan|Barat Daya|Barat Laut|Barat|Utara|Timur Laut|Kiblat|Khiblat)'
        elif entity_type == 'ownership':
            pattern = r'\b(SHM|Sertifikat Hak Milik|HGB|Hak Guna Bangunan)\b'
        else:
            pattern = rf'{re.escape(kata_kunci)}\s*[\:\.\-\s]*([\w\s\.,]+)'

        for sentence in ads_description:
            if isinstance(sentence, str) and kata_kunci.lower() in sentence.lower():
                match = re.search(pattern, sentence, re.IGNORECASE)
                if match:
                    if entity_type == 'electricity':
                        value_str = match.group(1).replace('.', '').replace(',', '')
                        return int(value_str)
                    elif entity_type in ['garage', 'carport']:
                        value_str = match.group(1)
                        return int(value_str) if value_str else 1
                    elif entity_type == 'heading':
                        heading = match.group(2).capitalize()
                        return 'Barat' if heading.lower() in ['kiblat', 'khiblat'] else heading
                    elif entity_type == 'ownership':
                        ownership = match.group(1).lower()
                        return "SHM" if "shm" in ownership or "sertifikat hak milik" in ownership else "HGB" if "hgb" in ownership or "hak guna bangunan" in ownership else None
                    else:
                        value_str = match.group(1).strip().split(" ")[0]
                        return value_str
    return None

# Function to count additional rooms
def count_rooms(ads_description, keyword_list):
    for keyword in keyword_list:
        pattern = rf'(\d+)?\s*{re.escape(keyword)}'
        for sentence in ads_description:
            match = re.search(pattern, sentence, re.IGNORECASE)
            if match:
                return int(match.group(1)) if match.group(1) else 1
    return 0

# Function to extract number of floors
def extract_floors(ads_description, keyword_list):
    for keyword in keyword_list:
        pattern = rf'(\d+[,.]?\d*)\s*{re.escape(keyword)}|{re.escape(keyword)}\s*[\:\-\s]*(\d+[,.]?\d*)'
        for sentence in ads_description:
            match = re.search(pattern, sentence, re.IGNORECASE)
            if match:
                number = match.group(1) if match.group(1) else match.group(2)
                return float(number.replace(',', '.')) if number else 1
    return 1

# Convert floors column to float
df['floors'] = df['floors'].astype(float)

# Iterate over dataset and update only missing values
for index, row in tqdm(df.iterrows(), total=len(df), desc="Processing rows"):
    description = eval(row['description']) if isinstance(row['description'], str) else row['description']
    property_type = row['type']
    
    facilities = row['facilities']

    if not isinstance(description, list):
        description = [description] if isinstance(description, str) else []
    
    # Convert facilities column to a list if it's not already
    if isinstance(facilities, str):
        try:
            facilities = eval(facilities)  # Convert string representation of list to an actual list
        except:
            facilities = [facilities]
    elif not isinstance(facilities, list):
        facilities = []

    # Update electricity capacity if missing
    if pd.isna(row['electricity_capacity']) or row['electricity_capacity'] == "":
        df.at[index, 'electricity_capacity'] = scrape_description(description, ['Listrik', 'Daya Listrik'], entity_type='electricity')

    # Process garage capacity
    if row['garage_capacity'] == 0:
        garage_capacity = scrape_description(description, ['Garasi'], entity_type='garage')
        if garage_capacity is None and "Garasi" in facilities:
            garage_capacity = 1  # If "Garasi" is found in facilities, set to 1
        df.at[index, 'garage_capacity'] = garage_capacity if garage_capacity is not None else 0  # Keep default if no data found

    # Process carport capacity
    if row['carport_capacity'] == 0:
        carport_capacity = scrape_description(description, ['Carport'], entity_type='carport')
        if carport_capacity is None and "Carport" in facilities:
            carport_capacity = 1  # If "Carport" is found in facilities, set to 1
        df.at[index, 'carport_capacity'] = carport_capacity if carport_capacity is not None else 0  # Keep default if no data found

    # Update house orientation if missing
    if pd.isna(row['house_orientation']) or row['house_orientation'] == "":
        df.at[index, 'house_orientation'] = scrape_description(description, ['Arah', 'Orientasi', 'Hadap'], entity_type='heading')
    
    # Update ownership type if missing
    if pd.isna(row['certificate']) or row['certificate'] == "":
        df.at[index, 'certificate'] = scrape_description(description, ['Hak Milik', 'Sertifikat', 'Hak', 'SHM', 'HGB'], entity_type='ownership')

    # Update additional rooms if missing
    if row['additional_rooms'] == 0.0:
        df.at[index, 'additional_rooms'] = 1 if count_rooms(description, ['Ruang', 'Kamar', 'Dapur', 'Gudang']) > 0 else 0

    if row['floors'] == 0.0:
        df.at[index, 'floors'] = 1 if property_type == "Apartemen" else extract_floors(description, ['Lantai'])

# Save the updated DataFrame
output_file = "updated_jabodetabeksur_olx_housing_dataset_.csv"
df.to_csv(output_file, index=False)

print(f"Updated CSV saved to {output_file}")

Processing rows: 100%|██████████| 16148/16148 [00:05<00:00, 2890.66it/s]


Updated CSV saved to updated_jabodetabeksur_olx_housing_dataset_.csv


```Markdown
# Second Step of Extracting Information

This step going to extract information from the description again to find a more relevant information for the profile matching step

## List of Information Needed
1. Is it free from flooding?
2. Is it near public transportation (LRT/MRT/TJ)
3. Is it near highway? (Car oriented development)
4. Is it near any educational facilities? (TK/SD/SMP/SMA)
5. Is it near or in the heart of the city? (Might need some visualisation of the given district to determine this)

## List of Possible New Column
1. TK
2. SD
3. SMP
4. SMP
5. UNIV
6. Public Transport
7. Pusat Perbelajaan
8. Rumah Sakit

## Assign a Strategic Score 
```

In [26]:
import pandas as pd
import folium
from geopy.geocoders import Nominatim
from tqdm import tqdm

# Initialize geolocator
geolocator = Nominatim(user_agent="geoapiExercises")

# Function to get latitude & longitude
def get_coordinates(subdistrict):
    try:
        location = geolocator.geocode(f"{subdistrict}, Indonesia")
        if location:
            return location.latitude, location.longitude
    except:
        return None
    return None

# Apply geocoding to get lat/lon for each unique subdistrict
tqdm.pandas()
df[['latitude', 'longitude']] = df['address_subdistrict'].progress_apply(lambda x: pd.Series(get_coordinates(x)))

# Drop rows where coordinates could not be found
df.dropna(subset=['latitude', 'longitude'], inplace=True)

# Create a Folium map centered at Indonesia
m = folium.Map(location=[-6.1751, 106.8650], zoom_start=10)  # Centered near Jakarta

# Add markers to the map
for _, row in df.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=f"{row['address_subdistrict']}",
        icon=folium.Icon(color="blue", icon="info-sign")
    ).add_to(m)

# Save map to an HTML file
m.save("subdistrict_map.html")


100%|██████████| 16148/16148 [13:26:41<00:00,  3.00s/it]  


ValueError: Columns must be same length as key